In [ ]:
import pandas as pd
import sys,os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

os.chdir('C:/Users/arpit.goel/Documents/Projects/Kaggle/24_RecommendationDesign')
df_train=pd.read_csv('01.RawData/train.csv')
df_test=pd.read_csv('01.RawData/test.csv')
df_chal=pd.read_csv('01.RawData/challenge_data.csv')
df_sample=pd.read_csv('01.RawData/sample_submission_J0OjXLi_DDt3uQN.csv')
df_chal['publish_date']=pd.to_datetime(df_chal['publish_date'],format='%d-%m-%Y')
df_chal.sort_values(by='publish_date',inplace=True)
df_chal['challenge_id']=np.arange(len(df_chal))

users=df_train[['user_id']].drop_duplicates()
np.random.seed(1234)
users['dtype']=np.where(np.random.rand(len(users))>0.75,'valid','train')
users=users.set_index('user_id')['dtype']
df_train['ds']=df_train['user_id'].map(users)
df_test['ds']='test'
master=pd.merge(pd.concat([df_train,df_test]),df_chal,left_on=['challenge'],right_on=['challenge_ID'])
master.sort_values(by=['user_id','challenge_sequence'],inplace=True)


In [3]:
import pandas as pd
import sys,os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

os.chdir('C:/Users/arpit.goel/Documents/Projects/Kaggle/24_RecommendationDesign')
%run 00.Scripts/02.FreqItemSets.py

(117928, 3)


00.Scripts/02.FreqItemSets.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  t1=pd.merge(group,freq_items,left_on=['challenge_id'],right_on=['challenge_id_x'])


1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000


In [ ]:
def get_datasegment(master,valid=True):
    if valid==True:
        a1=master[master['ds'].isin(['train'])]
        a2=master[master['ds'].isin(['valid'])]
        a3=a2[a2['challenge_sequence']<=10]
        a4=a2[a2['challenge_sequence']>10]
        return a1,a3,a4
    else:
        return master[master['ds'].isin(['train','valid'])],master[master['ds'].isin(['test'])],[]

train,test,target=get_datasegment(master,valid=True)


In [ ]:
target.head()

In [ ]:
def nmf(mat, latent_features,target=[],max_iter=100, error_limit=1e-6, fit_error_limit=1e-6):
    X=mat.fillna(0).values
    
    pred_index=mat.fillna(0).stack().to_frame('kda_ratio').reset_index()
    pred_index=pred_index['user_id'].astype(str)+'_'+pred_index['challenge_id'].astype(str)
    test=target.copy()
    test['íd']=test['user_id'].astype(str)+'_'+test['challenge_id'].astype(str)
    
    
    eps = 1e-5
    print 'Starting NMF decomposition with {} latent features and {} iterations.'.format(latent_features, max_iter)
    # mask
    mask = np.sign(X)

    # initial matrices. A is random [0,1] and Y is A\X.
    rows, columns = X.shape
    np.random.seed(1234)
    A = np.random.rand(rows, latent_features)
    A = np.maximum(A, eps)

    Y = linalg.lstsq(A, X)[0]
    Y = np.maximum(Y, eps)

    masked_X = mask * X
    X_est_prev = dot(A, Y)
    for i in range(1, max_iter + 1):
        # ===== updates =====
        # Matlab: A=A.*(((W.*X)*Y')./((W.*(A*Y))*Y'));
        top = dot(masked_X, Y.T)
        bottom = (dot((mask * dot(A, Y)), Y.T)) + eps
        A *= top / bottom

        A = np.maximum(A, eps)
        # print 'A',  np.round(A, 2)

        # Matlab: Y=Y.*((A'*(W.*X))./(A'*(W.*(A*Y))));
        top = dot(A.T, masked_X)
        bottom = dot(A.T, mask * dot(A, Y)) + eps
        Y *= top / bottom
        Y = np.maximum(Y, eps)
        # print 'Y', np.round(Y, 2)


        # ==== evaluation ====
        if i % 1 == 0 or i == 1 or i == max_iter:
            print 'Iteration {}:'.format(i),
            X_est = dot(A, Y)
            err = mask * (X_est_prev - X_est)
            fit_residual = np.sqrt(np.sum(err ** 2))
            X_est_prev = X_est

#             curRes = linalg.norm(mask * (X - X_est), ord='fro')
#             print 'fit residual', np.round(fit_residual, 4),
#             print 'total residual', np.round(curRes, 4),
#             if curRes < error_limit or fit_residual < fit_error_limit:
#                 break
#             # Validation error to find optimal number of iterations.
#             pred=pd.DataFrame(A.dot(Y)).stack().to_frame('kda_ratio')
#             pred.index=pred_index
#             test['prediction']=test['id'].map(pred['kda_ratio'])
#             test['error']=np.square(test['prediction']-test['kda_ratio'])
#             print 'test error', np.sqrt(test['error'].mean())
    return A,Y

# mat=pd.concat([train,test]).set_index(['user_id','challenge_id'])
# mat['dummy']=1
# mat=mat['dummy'].unstack()
nmf(mat,10,target)

In [ ]:
mat.shape

In [ ]:
pd.concat([train,test])['challenge_id'].value_counts().describe(percentiles=np.arange(0,100,10)/100.0)

In [ ]:
df_chal['total_submissions'].describe(percentiles=np.arange(0,100,10)/100.0)